In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 4

In [2]:
G_rwm2_4 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_4.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_4)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_4))

MultiDiGraph with 50000 nodes and 136350 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_4 = list(G_rwm2_4.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_4_df = nx.to_pandas_edgelist(G_rwm2_4) 
rwm2_4_df.head()

,source,target,weight
0,1P598YmCSnE7XU355yWpAvq6VKFr5VUn8B,1D2nANLeDsxmGJpyVxoVEusuiwKNcYJqY,1.0
1,13jLpsqyVi3nEk2Tv1FGRvBmMTSERwF16E,1HkpPDZf4UAJAKvFimMfCMi7iwvyVRLP6q,1.0
2,1PcUaPm1hWHwgPhrhHdeVtUd8avjRhGx5i,3Co1Na1kvHs77rYcq6a3ki7dH41cQbaD9k,1.0
3,13tQrW5XnDgWXdKasvjiCP8hXewK3Ckzmv,1H4ahHD6iQ9dc8HWA2HmggXgG5f9HuwMBi,1.0
4,13tQrW5XnDgWXdKasvjiCP8hXewK3Ckzmv,3Pfxj7hraySrByYj44iBeQg3YbbFVDnHZv,1.0


In [8]:
# Number of unique input addresses
rwm2_4_df['source'].nunique()

32001

In [9]:
# Number of unique output addresses
rwm2_4_df['target'].nunique()

31292

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_4_df.describe()

,weight
count,136350.000000
mean,0.776678
std,0.465861
min,-69.648093
25%,0.519019
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_4).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  293


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.59%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_4.degree()]
degree = [val for (node, val) in G_rwm2_4.degree()]
in_degree = [val for (node, val) in G_rwm2_4.in_degree()]
out_degree = [val for (node, val) in G_rwm2_4.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_4) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,1P598YmCSnE7XU355yWpAvq6VKFr5VUn8B,1,0,1,0
1,1BcH8kfqmSoWhXb5V3sH7hnk3rDBT82vEj,2,2,0,0
2,1MtSNa8LPkAs2Lcr2MQHg1wRdkCgyV64uM,1,1,0,0
3,13qAtDZ7QPWzkZ7gD2N6tEscmHnzDpoXp6,2,2,0,0
4,1FDc1jckp2YForxoToomKD7jSBKLZakDPK,3,3,0,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.454000,2.727000,2.727000
std,40.104142,18.037984,25.536991
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4330.000000,1707.000000,2623.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,293.000000,293.000000,293.000000
mean,133.945392,55.365188,78.580205
std,470.431167,195.877477,302.547570
min,1.000000,0.000000,0.000000
25%,5.000000,2.000000,1.000000
50%,22.000000,8.000000,7.000000
75%,75.000000,34.000000,30.000000
max,4330.000000,1707.000000,2623.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49707.000000,49707.000000,49707.000000
mean,4.696602,2.416722,2.279880
std,14.827666,9.244640,9.172761
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,834.000000,815.000000,689.000000


For all 3 cases the degree distribution properties are similar to the overall network.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_4) # Connectivity remains very low

5.4541090821816436e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_4 = k_clique_communities(G_rwm2_4.to_undirected(), k=10)
com_rwm2_4 = next(com_generator_rwm2_4)
print('The number of communities in the sample : ' + str(len(com_rwm2_4)))

The number of communities in the sample : 118


The sample has a large variety of community structures. (Note: Giant Component = 107)